# Ionization Fraction Tables

Create ASCII text tables of ion fractions for a given species in collisional ionization equilibrium (CIE).
Uses data from the [CHIANTI atomic database](https://www.chiantidatabase.org/) and the 
[ChiantiPy](https://github.com/chianti-atomic/ChiantiPy/) module to compute ionization fractions in CIE.

These files may be used as ancillary data for the ISM/IGM book if it is impractical to install CHIANTI
and ChiantiPy.  Tables have a brief comment block followed by a header that is readily parsed using pandas.

CHIANTI is a collaborative project involving George Mason University, the University of Michigan (USA),
University of Cambridge (UK) and NASA Goddard Space Flight Center (USA). 

In [ ]:
import math
import numpy as np

# Throttle nuisance warnings

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

# Chianti CIE code 

import ChiantiPy.core as ch

## Useful Definitions

### int2roman()

Function to convert an integer into a roman numeral string.

### species

List of the elements associated with nuclear charge Z-1 from hydrogen to nickel.


In [ ]:
def int2roman(integer):
    table = [['M',1000],['CM',900],['D',500],['CD',400],['C',100],
             ['XC',90],['L',50],['XL',40],['X',10],['IX',9],
             ['V',5],['IV',4],['I',1]]
    parts = []
    for letter, value in table:
        while value <= integer:
            integer -= value
            parts.append(letter)
    return ''.join(parts)

species = ['H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg','Al',
           'Si','P','S','Cl','Ar','K','Ca','Sc','Ti','V','Cr','Mn','Fe',
           'Co','Ni']

## Ionization Fractions Table

Change the nuclear charge Z to make a quick table of ionic fraction for each ionization stage of a species.

Prints a brief comment (#) block giving the provenance, a simple header of logT plus the roman numeral of
the ionic stage, and then creates an ASCII text file named 'ionFrac_x.txt', where x is the species 
(e.g., ionFrac_C.txt for carbon).

In this example we compute the ion fractions of Nitrogen (Z=7).

In [ ]:
# Nuclear charge

Z = 7

# Range of temperatures

minT = 9000. # K
maxT = 1.0e7 # K

minLogT = math.log10(minT)
maxLogT = math.log10(maxT)
dlogT = 0.01
numT = 1+int((maxLogT-minLogT)/dlogT)
logT = np.linspace(minLogT,maxLogT,num=numT)

T = 10.0**logT # temperature (K)

outFile = f'ionFrac_{species[Z-1]}.txt'
f = open(outFile,'w')

# put in a comment header

f.write(f'#\n')
f.write(f'# Ionization fraction vs T for {species[Z-1]}\n')
f.write(f'#   logT = {logT[0]:.2f} to {logT[-1]:.2f}\n')
f.write(f'# Calculated using the CHIANTI atomic database and ChiantiPy\n')
f.write(f'#\n')

cie = ch.ioneq(Z)
cie.load('chianti')  # default CHIANTI database
cie.calculate(T)

outStr = 'logT  '
for stage in range(Z+1):
    outStr += f'{int2roman(stage+1):6s} '
f.write(f'{outStr}\n')

for i in range(len(logT)):
    outStr = f'{logT[i]:.2f} '
    for j in range(Z+1):
        outStr += f'{cie.Ioneq[j][i]:.4f} '
    f.write(f'{outStr}\n')
    
f.close()

## How to read an ionFrac_x file

Code stub showing how to read in data from an ionFrac_x.txt file created with the code above using the pandas
read_csv() method.

In [ ]:
import pandas as pd

dataFile = 'ionFrac_x.txt'

# data file has a text header
#   logT, I, II, ... 

data = pd.read_csv(dataFile,sep=r'\s+',comment='#')

myLogT = np.array(data['logT'])
stageII = np.array(data['II'])
stageV = np.array(data['V'])